# Finance Combine Transactions: Fidelity

In [15]:
import pandas as pd
import os

In [16]:
# replace with your folder's path
# folder_path = r'/Users/ryanrunchey/Library/CloudStorage/SynologyDrive-On-demand/Documents/Finance/Accounts/Fidelity_7010_Hippo 401k_Brokerage Link/Transactions'
folder_path = r'/Users/ryanrunchey/Library/CloudStorage/SynologyDrive-On-demand/Documents/Finance/Accounts/Fidelity_7010_Hippo 401k_Brokerage Link/Transactions_All_Fidelity_Accounts'

all_files = os.listdir(folder_path)

# Filter out non-CSV files
csv_files = [f for f in all_files if f != 'Transactions_Fidelity_401k.csv' and f.endswith('.csv')]

# # Create a list to hold the dataframes
df_list = []

keep_cols = ['Run Date', 'Account', 'Action', 'Symbol', 'Security Description', 'Security Type', 'Quantity', 'Price ($)', 'Commission ($)', 'Fees ($)', 'Accrued Interest ($)', 'Amount ($)', 'Settlement Date']

df_concat = pd.concat([pd.read_csv(os.path.join(folder_path, csv))[keep_cols] for csv in csv_files], ignore_index=True)
df_concat.columns = [x.lower().replace(' ($)', '') for x in df_concat.columns]

df_concat['brokerage'] = 'Fidelity'

cols = ['run date', 'brokerage', 'account', 'action', 'symbol', 'security description', 'security type', 
        'quantity', 'price', 'commission', 'fees', 'accrued interest', 'amount', 'settlement date']

df_concat = df_concat[cols]

df_concat.columns = [x.replace(' ', '_') for x in df_concat.columns]

df_concat['run_date'] = pd.to_datetime(df_concat['run_date'], format='mixed')

df_concat

,run_date,brokerage,account,action,symbol,security_description,security_type,quantity,price,commission,fees,accrued_interest,amount,settlement_date
0,2023-12-29,Fidelity,HIPPO EMPLOYEE SVCE 25297,Dividend,NaN,FID GOVT MMKT K6,NaN,20.570,NaN,NaN,NaN,NaN,20.57,NaN
1,2023-12-29,Fidelity,BrokerageLink 652967010,YOU BOUGHT SPDR BLOOMBERG 1-3 MONTH T-BILL ET...,BIL,SPDR BLOOMBERG 1-3 MONTH T-BILL ETF,Cash,0.320,91.40,NaN,NaN,NaN,-29.25,1/3/24
2,2023-12-29,Fidelity,BrokerageLink 652967010,YOU BOUGHT SPDR BLOOMBERG 1-3 MONTH T-BILL ET...,BIL,SPDR BLOOMBERG 1-3 MONTH T-BILL ETF,Cash,39.000,91.39,NaN,NaN,NaN,-3564.40,1/3/24
3,2023-12-29,Fidelity,BrokerageLink 652967010,YOU SOLD ISHARES TRUST MSCI EMG MKTS ETF USD ...,EEM,ISHARES TRUST MSCI EMG MKTS ETF USD DIS,Cash,-64.000,40.31,NaN,0.03,NaN,2579.83,1/3/24
4,2023-12-29,Fidelity,BrokerageLink 652967010,YOU SOLD ISHARES TRUST MSCI EMG MKTS ETF USD ...,EEM,ISHARES TRUST MSCI EMG MKTS ETF USD DIS,Cash,-0.197,40.31,NaN,NaN,NaN,7.94,1/3/24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,2024-01-16,Fidelity,BrokerageLink,REINVESTMENT as of Jan-15-2024 INVESCO QQQ TR ...,QQQ,INVESCO QQQ TR UNIT SER 1,Cash,0.018,407.39,NaN,NaN,NaN,-7.34,NaN
982,2024-01-16,Fidelity,BrokerageLink,DIVIDEND RECEIVED as of Jan-15-2024 INVESCO QQ...,QQQ,INVESCO QQQ TR UNIT SER 1,Cash,0.000,NaN,NaN,NaN,NaN,7.34,NaN
983,2024-01-16,Fidelity,BrokerageLink,TRANSFERRED FROM TO BROKERAGE OPTION (Cash),,No Description,Cash,0.000,NaN,NaN,NaN,NaN,3197.12,NaN
984,2024-01-12,Fidelity,HIPPO EMPLOYEE SVCE,Contributions,NaN,FID GOVT MMKT K6,NaN,168.270,NaN,NaN,NaN,NaN,168.27,NaN


In [17]:
# Google BigQuery Authentication

from google.cloud import bigquery
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Export data

from pandas_gbq import to_gbq
import pickle

# Set your OAuth client ID JSON downloaded from GCP Console
# CLIENT_SECRET_FILE = 'client_secret.json'  # downloaded from GCP
CLIENT_SECRET_FILE = os.path.expanduser("/Users/ryanrunchey/credentials/gcp_credentials/client_secret_295707256455-0fsr3bqoc89psl22fgp2cfipbd4m1s1v.apps.googleusercontent.com.json")
SCOPES = ['https://www.googleapis.com/auth/cloud-platform']

# Authenticate interactively (stores a token locally for reuse)
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)
else:
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
    credentials = flow.run_local_server(port=0)
    with open('token.pickle', 'wb') as token:
        pickle.dump(credentials, token)

# Initialize the BigQuery client with those credentials
client = bigquery.Client(credentials=credentials, project="ryanrunchey")

In [18]:
# Test query
query = """
SELECT
  underlying_symbol,
  symbol,
  SUM(quantity) AS quantity,
  SUM(fees_and_commissions) AS fees_and_commissions,
  SUM(amount) AS amount,
  SAFE_DIVIDE(SUM(amount), SUM(quantity)) AS net_price_per_unit
FROM
  ryanrunchey.account_transactions.fct_transactions
WHERE
  underlying_symbol = 'ENVX'
GROUP BY
  1,2
ORDER BY
  1,2
"""

df = client.query(query).to_dataframe()
df

,underlying_symbol,symbol,quantity,fees_and_commissions,amount,net_price_per_unit
0,ENVX,None,0E-9,None,40.660000000,None
1,ENVX,-ENVX231117C15,0E-9,2.720000000,345.280000000,None
2,ENVX,-ENVX240119C10,0E-9,2.680000000,-152.680000000,None
3,ENVX,-ENVX240119C17.5,0E-9,2.730000000,529.190000000,None
4,ENVX,-ENVX240119C20,0E-9,1.360000000,258.600000000,None
5,ENVX,-ENVX240719C15,0E-9,4.020000000,-169.020000000,None
6,ENVX,-ENVX250117C12.5,0E-9,6.740000000,3538.990000000,None
7,ENVX,-ENVX250117C20,0E-9,10.090000000,3348.510000000,None
8,ENVX,-ENVX260116C8,-36.000000000,24.500000000,13257.500000000,-368.263888889
9,ENVX,6734859EC,0E-9,5.410000000,1911.590000000,None


In [19]:
# Export to csv
df_concat.to_csv(os.path.join(folder_path, 'Transactions_Fidelity_401k.csv'), index=False)

# Export to pickle
df_concat.to_pickle(os.path.join(folder_path, 'Transactions_Fidelity_401k.csv'))

# Export to BigQuery
to_gbq(
    dataframe=df_concat,
    destination_table="account_transactions.fidelity_401k",
    project_id="ryanrunchey",
    if_exists="replace"  # or "append"
)

100%|██████████| 1/1 [00:00<00:00, 9510.89it/s]


## Investigation

In [22]:
mask = df_concat['symbol'].str.contains('-ENVX1260116C8', na=False)
df_concat[mask]

,run_date,brokerage,account,action,symbol,security_description,security_type,quantity,price,commission,fees,accrued_interest,amount,settlement_date
542,2025-07-23,Fidelity,BrokerageLink,YOU BOUGHT CLOSING TRANSACTION CALL ENOVIX COR...,-ENVX1260116C8,CALL ENOVIX CORPORAT 100ENVX+14ENVXW JAN 16 26...,Cash,5.0,7.75,3.25,0.13,NaN,-3878.38,7/24/25
543,2025-07-23,Fidelity,BrokerageLink,YOU BOUGHT CLOSING TRANSACTION CALL ENOVIX COR...,-ENVX1260116C8,CALL ENOVIX CORPORAT 100ENVX+14ENVXW JAN 16 26...,Cash,5.0,7.85,3.25,0.13,NaN,-3928.38,7/24/25
544,2025-07-23,Fidelity,BrokerageLink,YOU BOUGHT CLOSING TRANSACTION CALL ENOVIX COR...,-ENVX1260116C8,CALL ENOVIX CORPORAT 100ENVX+14ENVXW JAN 16 26...,Cash,5.0,8.00,3.25,0.13,NaN,-4003.38,7/24/25
545,2025-07-23,Fidelity,BrokerageLink,YOU BOUGHT CLOSING TRANSACTION CALL ENOVIX COR...,-ENVX1260116C8,CALL ENOVIX CORPORAT 100ENVX+14ENVXW JAN 16 26...,Cash,15.0,7.75,9.75,0.40,NaN,-11635.15,7/24/25
555,2025-07-21,Fidelity,BrokerageLink,YOU BOUGHT CLOSING TRANSACTION CALL ENOVIX COR...,-ENVX1260116C8,CALL ENOVIX CORPORAT 100ENVX+14ENVXW JAN 16 26...,Cash,1.0,7.65,0.65,0.03,NaN,-765.68,7/22/25
558,2025-07-17,Fidelity,BrokerageLink,DISTRIBUTION WARRANT DISTRIBUTION CALL ENOVIX ...,-ENVX1260116C8,CALL ENOVIX CORPORAT 100ENVX+14ENVXW JAN 16 26...,Shares,-31.0,NaN,NaN,NaN,NaN,-25947.00,NaN


In [23]:
mask = df_concat['symbol'].str.contains('6734859EC', na=False)
df_concat[mask]

,run_date,brokerage,account,action,symbol,security_description,security_type,quantity,price,commission,fees,accrued_interest,amount,settlement_date
559,2025-07-17,Fidelity,BrokerageLink,DISTRIBUTION WARRANT DISTRIBUTION CALL (ENVX) ...,6734859EC,CALL (ENVX) ENOVIX CORPORATION JAN 16 26 $25 (...,Shares,8.0,NaN,NaN,NaN,NaN,1064.00,NaN
563,2025-07-10,Fidelity,BrokerageLink,YOU SOLD OPENING TRANSACTION CALL (ENVX) ENOVI...,6734859EC,CALL (ENVX) ENOVIX CORPORATION JAN 16 26 $25 (...,Cash,-5.0,1.07,3.25,0.13,NaN,531.62,7/11/25
564,2025-07-10,Fidelity,BrokerageLink,YOU SOLD OPENING TRANSACTION CALL (ENVX) ENOVI...,6734859EC,CALL (ENVX) ENOVIX CORPORATION JAN 16 26 $25 (...,Cash,-3.0,1.06,1.95,0.08,NaN,315.97,7/11/25


In [24]:
mask = df_concat['symbol'].str.contains('-ENVX1260116C25', na=False)
df_concat[mask]

,run_date,brokerage,account,action,symbol,security_description,security_type,quantity,price,commission,fees,accrued_interest,amount,settlement_date
557,2025-07-17,Fidelity,BrokerageLink,DISTRIBUTION WARRANT DISTRIBUTION CALL ENOVIX ...,-ENVX1260116C25,CALL ENOVIX CORPORAT 100ENVX+14ENVXW JAN 16 26...,Shares,-8.0,NaN,NaN,NaN,NaN,-1120.0,NaN
